In [1]:
import pandas as pd
import numpy as py
import pyodbc

sql_conn = pyodbc.connect("""DRIVER={SQL Server};
                            server=;
                            database=;
                            uid=;
                            pwd=""")
query = """EXEC SP_Contratos_Por_Vencer

Select *
From [Email].[dbo].[Reporte_Vencimiento]"""
df = pd.read_sql(query, sql_conn)
df.head()

,SUCVTA,SUCCOL,REGIONAL,Empresa_Emisora,ANEXO,INDICE,SITIO,INDICE1,AREA,TIPO,...,CLIENTE,CLIENTEEXHIBE,AGENCIA,COMAG,INICTTO,FINCTTO,INIEXH,FINEXH,Fecha_Conv,Vencimiento
0,MEXICO,MERIDA,CDMX,None,2144,17,NaN,None,NaN,AUTOBUSES,...,AMX CONTENIDO SA DE CV,TELMEX,JV VALE SA DE CV,15.0,2019-02-26,2019-04-25,2019-03-12,2019-05-11,2019-02-25,<30
1,MEXICO,MERIDA,CDMX,None,2144,18,NaN,None,NaN,AUTOBUSES,...,AMX CONTENIDO SA DE CV,TELMEX,JV VALE SA DE CV,15.0,2019-02-26,2019-04-25,2019-03-19,2019-05-18,2019-02-25,<30
2,MEXICO,MERIDA,CDMX,None,2144,19,NaN,None,NaN,AUTOBUSES,...,AMX CONTENIDO SA DE CV,TELMEX,JV VALE SA DE CV,15.0,2019-02-26,2019-04-25,2019-03-12,2019-05-11,2019-02-25,<30
3,MONTERREY,MEXICO,NORESTE,None,2160,1,NaN,None,NaN,AUTOBUSES,...,"WHIRLPOOL MEXICO, S.A. DE C.V.",WHIRLPOOL,PROMOTORA DE MEDIOS EFECTIVOS SA DE CV,15.0,2019-04-01,2019-06-30,2019-04-24,2019-07-23,2019-03-21,60-90
4,MONTERREY,MEXICO,NORESTE,None,2160,2,NaN,None,NaN,AUTOBUSES,...,"WHIRLPOOL MEXICO, S.A. DE C.V.",WHIRLPOOL,PROMOTORA DE MEDIOS EFECTIVOS SA DE CV,15.0,2019-04-01,2019-06-30,2019-04-24,2019-07-23,2019-03-21,60-90


In [2]:
regionales = df['REGIONAL'].unique().tolist()
regionales.append('TODAS')
regionales

['CDMX', 'NORESTE', 'GUADALAJARA', 'BAJIO', 'NOROESTE', 'SURESTE', 'TODAS']

In [ ]:
import smtplib 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders 


SERVER = "test"
FROM = "test"
TO = [{'CDMX':"test@test.com", 'NORESTE':"test@test.com", 'GUADALAJARA':"test@test.com", 'BAJIO':"test@test.com", 'NOROESTE':"test@test.com", 'SURESTE':"test@test.com", 'TODAS':"test@test.com"}]

for x in regionales:
    if x != 'TODAS':
        df[df.REGIONAL == x].to_csv(f'Regional_{x}.csv', index=False)
    else:
        df.to_csv(f'Regional_{x}.csv', index=False)
    # instance of MIMEMultipart 
    msg = MIMEMultipart() 
  
    # storing the senders email address   
    msg['From'] = FROM
  
    # storing the receivers email address  
    msg['To'] = TO[0][x]
  
    # storing the subject  
    msg['Subject'] = "Reporte de Vencimiento de Contratos"
  
    # string to store the body of the mail 
    body = "Hola, buen dia equipo! \n \n En el archivo adjunto se listan los sitios que están próximos a vencer ( 30,60 y 90 días ), para su revisión y acciones de forma oportuna. \n \n Cualquier duda o comentario por favor revisarlo con Gabriela Gonzalez al correo Gabriela.Gonzalezr@jcdecaux.com \n \n Saludos."


    # attach the body with the msg instance
    msg.attach(MIMEText(body, 'plain')) 

    # open the file to be sent  
    filename = f"Regional_{x}.csv"
    attachment = open(f"Regional_{x}.csv", "rb") 


    # instance of MIMEBase and named as p 
    p = MIMEBase('application', 'octet-stream') 
  
    # To change the payload into encoded form 
    p.set_payload((attachment).read()) 
  
    # encode into base64 
    encoders.encode_base64(p) 
   
    p.add_header('Content-Disposition', "attachment; filename= %s" % filename)


    # attach the instance 'p' to instance 'msg' 
    msg.attach(p) 
  



    server = smtplib.SMTP(SERVER, port)
    server.connect(SERVER, port)
    server.ehlo()
    server.starttls()
    server.ehlo()
    server.login("test@test.com", "PWD")
    text = msg.as_string() 
    server.sendmail(FROM, TO[0][x].split(","), text)
    server.quit()